In [ ]:
import os
import gc
import random

import numpy as np
from PIL import Image
from tqdm import tqdm 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage import img_as_float
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.optimizers import adam
from keras.utils import conv_utils
from keras.engine.topology import Layer
from keras.datasets import mnist, cifar10
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, PReLU, BatchNormalization, Add, LeakyReLU, Dense, Flatten
from keras.layers.merge import Concatenate

In [ ]:
path_data = '/home/ms3u14/COMP6208-AutoEncoder/several27/data/'
path_div2k_train_hr = path_data + 'DIV2K_train_HR/'
path_div2k_train_lr = path_data + 'DIV2K_train_LR_bicubic/X4/'
path_div2k_test_hr = path_data + 'DIV2K_valid_HR/'
path_div2k_test_lr = path_data + 'DIV2K_valid_LR_bicubic/X4/'

# Data preprocessing

Other useful dataset: 
- https://data.vision.ee.ethz.ch/cvl/DIV2K/

## Load

In [ ]:
def norm(x):
    return (x - 0.5) / 0.5

def denorm(x):
    return x * 0.5 + 0.5

def norm_to_img(x):
    return Image.fromarray((denorm(x) * 255).astype(np.uint8))

def img_to_norm(x):
    return norm(img_as_float(x))

In [ ]:
x_train = []
with tqdm() as progress:
    for file in os.listdir(path_div2k_train_hr):
        if file.endswith('png'):
            x_train.append(img_to_norm(Image.open(path_div2k_train_hr + file)))
            
        progress.update()

In [ ]:
counter = {}
for x in x_train:
    counter.setdefault(x.shape[0], 0)
    counter[x.shape[0]] += 1

print(counter)

In [ ]:
x_train_norm = (img_as_float(x_train[0]) - 0.5) / 0.5

In [ ]:
norm_to_img(x_train_norm)

## Display sample 

In [ ]:
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow((x_train_norm * x_train_std + x_train_mean).astype(np.uint8)[i])
    
plt.show()

## Load CIFAR

In [ ]:
def generator_images(path, size=(420, 560, 3), ratio=2, batch_size=32):
    lr_height, lr_width = size[0] // ratio, size[1] // ratio
    
    batch_i = 0
    batch = np.zeros((batch_size, size[0], size[1], size[2]))
    batch_scaled = np.zeros((batch_size, lr_height, lr_width, size[2]))
    
    while True:
        for file in os.listdir(path):
            if not file.endswith('.jpg'):
                continue 
            
            if batch_i == batch_size:
                yield batch_scaled, batch
                
                batch_i = 0
                batch = np.zeros((batch_size, size[0], size[1], size[2]))
                batch_scaled = np.zeros((batch_size, lr_height, lr_width, size[2]))
            
            file_path = path + file
            img = img_as_float(Image.open(file_path))
            if len(img.shape) == 2: 
                 img = np.asarray(np.dstack((img, img, img))) 
            
            batch[batch_i] = img
            batch_scaled[batch_i] = resize(img, (lr_height, lr_width))
            
            batch_i += 1

In [ ]:
def count_images(path):
    return sum([1 for file in os.listdir(path) if file.endswith('.jpg')])

In [ ]:
count_images(path_open_images_560_420_train), count_images(path_open_images_560_420_val)

# Network description 

![](srgan_architecture.png)

Useful links: 
- https://github.com/liangstein/SRGAN-Keras/blob/master/py.py
- https://github.com/titu1994/Super-Resolution-using-Generative-Adversarial-Networks/blob/master/models.py
- https://github.com/soumith/ganhacks

# Network implementation 

## Generator network

1. Input - Image LR
2. Conv, kernel 3x3, 64 feature maps, what's s1?
3. ParametricReLU
4. B=16 residual blocks
    1. Conv with 3x3 kernel, 64 feature maps and stride 1
    2. Batch Normalization (https://keras.io/layers/normalization/)
    3. ParametricReLU
    4. Conv with 3x3 kernel, 64 feature maps and stride 1
    5. Batch Normalization
    6. Elementwise sum (https://keras.io/layers/merge/ add)
5. Conv with 3x3 kernel, 64 feature maps and stride 1
6. Batch Normalization
7. Elementwise sum (https://keras.io/layers/merge/ add)
8. Shuffle block x2 (? how it's called ?)
    1. Conv with 3x3 kernel, 256 feature maps and stride 1
    2. PixelShuffler x2 https://gist.github.com/t-ae/6e1016cc188104d123676ccef3264981
    3. ParametricReLU
9. Conv with 9x9 kernel, 3 feature maps and stride 1
10. Estimated SR (super-resolved) image

Is the padding `same` a correct one? Otherwise the dims are not the same
Is the number of pixel shufflers the scaling ratio? 

In [ ]:
# from https://gist.github.com/t-ae/6e1016cc188104d123676ccef3264981

class PixelShuffler(Layer):
    def __init__(self, size=(2, 2), data_format=None, **kwargs):
        super(PixelShuffler, self).__init__(**kwargs)
        self.data_format = conv_utils.normalize_data_format(data_format)
        self.size = conv_utils.normalize_tuple(size, 2, 'size')

    def call(self, inputs):

        input_shape = K.int_shape(inputs)
        if len(input_shape) != 4:
            raise ValueError('Inputs should have rank ' +
                             str(4) +
                             '; Received input shape:', str(input_shape))

        if self.data_format == 'channels_first':
            batch_size, c, h, w = input_shape
            if batch_size is None:
                batch_size = -1
            rh, rw = self.size
            oh, ow = h * rh, w * rw
            oc = c // (rh * rw)

            out = K.reshape(inputs, (batch_size, rh, rw, oc, h, w))
            out = K.permute_dimensions(out, (0, 3, 4, 1, 5, 2))
            out = K.reshape(out, (batch_size, oc, oh, ow))
            return out

        elif self.data_format == 'channels_last':
            batch_size, h, w, c = input_shape
            if batch_size is None:
                batch_size = -1
            rh, rw = self.size
            oh, ow = h * rh, w * rw
            oc = c // (rh * rw)

            out = K.reshape(inputs, (batch_size, h, w, rh, rw, oc))
            out = K.permute_dimensions(out, (0, 1, 3, 2, 4, 5))
            out = K.reshape(out, (batch_size, oh, ow, oc))
            return out

    def compute_output_shape(self, input_shape):

        if len(input_shape) != 4:
            raise ValueError('Inputs should have rank ' +
                             str(4) +
                             '; Received input shape:', str(input_shape))

        if self.data_format == 'channels_first':
            height = input_shape[2] * self.size[0] if input_shape[2] is not None else None
            width = input_shape[3] * self.size[1] if input_shape[3] is not None else None
            channels = input_shape[1] // self.size[0] // self.size[1]

            if channels * self.size[0] * self.size[1] != input_shape[1]:
                raise ValueError('channels of input and size are incompatible')

            return (input_shape[0],
                    channels,
                    height,
                    width)

        elif self.data_format == 'channels_last':
            height = input_shape[1] * self.size[0] if input_shape[1] is not None else None
            width = input_shape[2] * self.size[1] if input_shape[2] is not None else None
            channels = input_shape[3] // self.size[0] // self.size[1]

            if channels * self.size[0] * self.size[1] != input_shape[3]:
                raise ValueError('channels of input and size are incompatible')

            return (input_shape[0],
                    height,
                    width,
                    channels)

    def get_config(self):
        config = {'size': self.size,
                  'data_format': self.data_format}
        base_config = super(PixelShuffler, self).get_config()

        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
def srgan_generator(input_shape, input_=None):
    kernel_size = (3, 3)
    kernel_size_last = (9, 9)
    features = 64
    features_shuffle = 256
    features_last = 3
    B = 16

    # 1, 2, 3
    input_1 = input_ if input_ is not None else Input(shape=input_shape)
    conv2d_2 = Conv2D(filters=features, kernel_size=kernel_size, strides=(1, 1), padding='same', 
                      name='generator_conv2d_2')(input_1)
    prelu_3 = PReLU(name='generator_prelu_3')(conv2d_2)

    # 4 - residual blocks
    last_layer = prelu_3
    for i in range(B):
        conv2d_4_A = Conv2D(filters=features, kernel_size=kernel_size, strides=(1, 1), padding='same', 
                            name='generator_conv2d_4_A_%s' % i)(last_layer)
        bn_4_B = BatchNormalization(name='generator_bn_4_B_%s' % i)(conv2d_4_A)
        prelu_4_C = PReLU(name='generator_prelu_4_C_%s' % i)(bn_4_B)
        conv2d_4_D = Conv2D(filters=features, kernel_size=kernel_size, strides=(1, 1), padding='same', 
                            name='generator_conv2d_4_D_%s' % i)(prelu_4_C)
        bn_4_E = BatchNormalization(name='generator_bn_4_E_%s' % i)(conv2d_4_D)
        add_4_F = Add(name='generator_add_4_F_%s' % i)([last_layer, bn_4_E])

        last_layer = add_4_F

    # 5, 6, 7
    conv2d_5 = Conv2D(filters=features, kernel_size=kernel_size, strides=(1, 1), padding='same', 
                      name='generator_conv2d_5')(last_layer)
    bn_6 = BatchNormalization(name='generator_bn_6')(conv2d_5)
    add_7 = Add(name='generator_add_7')([prelu_3, bn_6])

    # 8 - shuffle block
    last_layer = add_7
    for i in range(1):
        conv2d_8_A = Conv2D(filters=features_shuffle, kernel_size=kernel_size, strides=(1, 1), padding='same', 
                            name='generator_conv2d_8_A_%s' % i)(last_layer)
        shuffler_8_B = PixelShuffler(name='generator_shuffler_8_B_%s' % i)(conv2d_8_A)
        prelu_8_C = PReLU(name='generator_prelu_8_C_%s' % i)(shuffler_8_B)

        last_layer = prelu_8_C

    # 9 
    conv2d_9 = Conv2D(filters=features_last, kernel_size=kernel_size_last, strides=(1, 1), 
                      padding='same', activation='tanh', name='generator_conv2d_9')(last_layer)

    return Model(input_1, conv2d_9)

### Train generator

In [ ]:
model_generator = srgan_generator((x_train_lr.shape[1], x_train_lr.shape[2], x_train_lr.shape[3]))
# model_generator = srgan_generator((210, 280, 3))
model_generator.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_generator.summary()

In [ ]:
train_version = 5

In [ ]:
checkpointer = ModelCheckpoint(filepath='data/srgan_generator_weights_%s.{epoch:03d}_{val_acc:.4f}.hdf5' % train_version, 
                               verbose=1, save_best_only=False)
tb_callback = TensorBoard(log_dir='data/tensorboard/', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
model_generator.fit(x_train_lr_norm, x_train_norm, epochs=1, validation_data=(x_test_lr_norm, x_test_norm), 
                    callbacks=[tb_callback])

# batch_size = 1
# n_train = 30 # count_images(path_open_images_560_420_train)
# n_val = 3 # count_images(path_open_images_560_420_val)
# with tf.device('/gpu:0'):
#     model_generator.fit_generator(generator_images(path_open_images_560_420_train, batch_size=batch_size), 
#                                   steps_per_epoch=n_train // batch_size,
#                                   validation_data=generator_images(path_open_images_560_420_val, 
#                                                                    batch_size=batch_size),
#                                   validation_steps=n_val // batch_size, epochs=1, callbacks=[tb_callback])

In [ ]:
model_generator.save('data/srgan_generator_%s.model' % train_version)

In [ ]:
model_generator.load_weights('data/srgan_generator_%s.model' % train_version)

In [ ]:
for g, type_ in [(generator_images(path_open_images_560_420_train, batch_size=1), 'train'), 
                 (generator_images(path_open_images_560_420_val, batch_size=batch_size), 'test')]:
    plt.figure(type_, figsize=(15, 30))
    for i, (x_lr, x) in enumerate(g):
        x_lr_ = (x_lr[0] * 255).astype(np.uint8)
        x_ = (x[0] * 255).astype(np.uint8)

        plt.subplot(10, 3, (i * 3) + 1)
        plt.imshow(x_lr_)

        plt.subplot(10, 3, (i * 3) + 2)
        plt.imshow(x_)

        plt.subplot(10, 3, (i * 3) + 3)
        plt.imshow((model_generator.predict(x_lr)[0] * 255).astype(np.uint8))

        if i >= 9:
            break
            
    plt.savefig('data/srgan_generator_%s_%s.png' % (train_version, type_))
    plt.show()

In [ ]:
for x, x_lr, type_ in [(x_train, x_train_lr_norm, 'train'), (x_test, x_test_lr_norm, 'test')]:
    plt.figure(figsize=(15, 30))
    for i in range(10):
        plt.subplot(10, 3, (i * 3) + 1)
        plt.imshow(((x_lr[i] * x_train_lr_std + x_train_lr_mean) * 255).astype(np.uint8))

        plt.subplot(10, 3, (i * 3) + 2)
        plt.imshow(x[i])

        plt.subplot(10, 3, (i * 3) + 3)
        h, w = x_lr.shape[1], x_lr.shape[2]
        prediction = model_generator.predict(x_lr[i].reshape(1, h, w, 3))[0]
        plt.imshow((prediction * x_train_std + x_train_mean).astype(np.uint8))

    plt.savefig('data/srgan_generator_%s_%s.png' % (train_version, type_))
    plt.show()

In [ ]:
plt.figure(figsize=(15, 30))
for i in range(10):
    plt.subplot(10, 3, (i * 3) + 1)
    plt.imshow(x_test_lr[i])

    plt.subplot(10, 3, (i * 3) + 2)
    plt.imshow(x_test[i])

    plt.subplot(10, 3, (i * 3) + 3)
    w, h = x_test_lr.shape[1], x_test_lr.shape[2]
    plt.imshow(np.abs(model_generator.predict(x_test_lr[i].reshape(1, w, h, 3))[0]))

plt.savefig('data/srgan_generator_%s_test.png' % train_version)
plt.show()

## Discriminator network 

1. Input image
2. Conv2d with 3x3 kernel, 64 filters, and 1 stride
3. Leaky ReLU (https://keras.io/layers/advanced-activations/#leakyrelu)
3. Conv2d block
    1. Conv2d with 3x3 kernel, 64 filters, and 1 stride
    2. Batch normalization
    3. Leaky ReLU (https://keras.io/layers/advanced-activations/#leakyrelu)
4. Repeat 4 with 128 filters x 2 
5. Repeat 4 with 256 filters x 2 
6. Repeat 4 with 512 filters x 2 
7. Dense layer with 1024
8. Leaky ReLU
9. Dense 1 binary
10. Sigmoid

In [ ]:
def srgan_discriminator(input_prediction, input_original):
    input_shape = 32, 32, 3
    features_1 = 64
    features_2, features_3, features_4 = 128, 256, 512
    kernel_size = 3, 3
    strides = 1, 1
    strides_2 = 2, 2
    
    input_1 = Concatenate()([input_prediction, input_original])
        
    conv2d_2 = Conv2D(filters=features_1, kernel_size=kernel_size, strides=strides, padding='same', 
                      name='discriminator_conv2d_2')(input_1)
    lrelu_3 = LeakyReLU(name='discriminator_lrelu_3')(conv2d_2)
    
    conv2d_4_A = Conv2D(filters=features_1, kernel_size=kernel_size, strides=strides_2, padding='same', 
                        name='discriminator_conv2d_4_A')(lrelu_3)
    conv2d_4_B = BatchNormalization(name='discriminator_conv2d_4_B')(conv2d_4_A)
    lrelu_4_c = LeakyReLU(name='discriminator_lrelu_4_c')(conv2d_4_B)

    i = 0
    last_layer = lrelu_4_c
    for _features in [features_2, features_3, features_4]:
        for j in range(2):
            conv2d_5_A = Conv2D(filters=_features, kernel_size=kernel_size, strides=(strides if j == 0 else strides_2), 
                                padding='same', name='discriminator_conv2d_5_A_%s' % i)(last_layer)
            conv2d_5_B = BatchNormalization(name='discriminator_conv2d_5_B_%s' % i)(conv2d_5_A)
            lrelu_5_C = LeakyReLU(name='discriminator_lrelu_5_C_%s' % i)(conv2d_5_B)
            
            last_layer = lrelu_5_C
            i += 1
    
    flatten_8 = Flatten(name='discriminator_flatten_8')(last_layer)
    dense_8 = Dense(1024, name='discriminator_dense_8')(flatten_8)
    lrelu_9 = LeakyReLU(name='discriminator_lrelu_9')(dense_8)
    dense_10 = Dense(1, activation='sigmoid', name='discriminator_dense_10')(lrelu_9)
    
    return Model([input_prediction, input_original], dense_10)

### Train discriminator

In [ ]:
def discriminator_generator_cifar(model_generator, x, x_predictions, batch_size=64):        
    batch_size = batch_size // 2
    
    batch_i = 0
    batch = np.zeros((batch_size, 32, 32, 3))
    batch_predictions = np.zeros((batch_size, 32, 32, 3))
    
    while True:
        for i in range(x.shape[0]):
            if batch_i == batch_size:
                
                # TODO: use soft and noisy labels
                yield [np.concatenate((batch_predictions, batch)), np.concatenate((batch, batch))], \
                    np.concatenate((np.zeros(batch_size), np.ones(batch_size)))
                
                batch_i = 0
                batch = np.zeros((batch_size, 32, 32, 3))
                batch_predictions = np.zeros((batch_size, 32, 32, 3))
            
            batch[batch_i] = x[i]
            batch_predictions[batch_i] = x_predictions[i]
            
            batch_i += 1

In [ ]:
epochs = 5
batch_size = 64
train_version = 5

In [ ]:
# generator_in = Input(shape=(16, 16, 3))

model_generator = srgan_generator((16, 16, 3)) #, input_=generator_in)

discriminator_input_prediction = Input(shape=(32, 32, 3))
discriminator_input_original = Input(shape=(32, 32, 3))
model_discriminator = srgan_discriminator(discriminator_input_prediction, discriminator_input_original)

# generator_out = model_generator(generator_in)

# discriminator_out = model_discriminator(generator_out)
# model_srgan = Model(generator_in, discriminator_out)

In [ ]:
model_generator.compile(optimizer='adam', loss='mse')
model_discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# TODO try using softmax instead of sigmoid for categorical crossentropy
# model_srgan.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_discriminator.summary()

In [ ]:
model_generator.load_weights('data/srgan_generator_%s.model' % train_version)

In [ ]:
path_train_predictions = path_data + ('srgan_generator_%s_train_predictions.npy' % train_version)
path_test_predictions = path_data + ('srgan_generator_%s_test_predictions.npy' % train_version)

In [ ]:
x_train_predictions = model_generator.predict(x_train_lr_norm)
x_test_predictions = model_generator.predict(x_test_lr_norm)

In [ ]:
np.min(x_train_predictions), np.max(x_train_predictions)

In [ ]:
# TEMPORARY FIX OF GPU MEMORY LEAK ? 

np.save(path_train_predictions, x_train_predictions)
np.save(path_test_predictions, x_test_predictions)

In [ ]:
x_train_predictions = np.load(path_train_predictions)
x_test_predictions = np.load(path_test_predictions)

In [ ]:
checkpointer = ModelCheckpoint(filepath='data/srgan_discriminator_weights_%s.{epoch:03d}_{val_acc:.4f}.hdf5' % 
                               train_version, verbose=1, save_best_only=False)
tb_callback = TensorBoard(log_dir='data/tensorboard/', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
# model_generator.fit(x_train_lr, x_train, epochs=100, validation_data=(x_test_lr, x_test), callbacks=[tb_callback])

with tf.device('/gpu:0'):
    model_discriminator.fit_generator(discriminator_generator_cifar(model_generator, x_train_norm, x_train_predictions, 
                                                                    batch_size), 
                                      steps_per_epoch=x_train.shape[0] // (batch_size * 2), 
                                      validation_data=discriminator_generator_cifar(model_generator, 
                                                                                    x_test_norm, x_test_predictions, 
                                                                                    batch_size), 
                                      validation_steps=x_test.shape[0] // (batch_size * 2), epochs=epochs, 
                                      callbacks=[tb_callback, checkpointer])

In [ ]:
model_discriminator.save('data/srgan_discriminator_%s.model' % train_version)

In [ ]:
model_discriminator.predict([model_generator.predict(x_train_lr_norm[2].reshape(1, 16, 16, 3)),
                             x_train_norm[2].reshape(1, 32, 32, 3)])

## Train full SRGAN

In [ ]:
# https://github.com/titu1994/Super-Resolution-using-Generative-Adversarial-Networks/blob/master/models.py#L420
# What shall I do? 

# generator_in = Input(shape=(16, 16, 3))
# discriminator_in = Input(shape=(32, 32, 16, 3))

# model_generator = srgan_generator((16, 16, 3), input_=generator_in)
# model_discriminator = srgan_discriminator()

# generator_out = model_generator(generator_in)

# discriminator_out = model_discriminator(generator_out)
# model_srgan = Model(generator_in, discriminator_out)

generator_in = Input(shape=(16, 16, 3))
model_generator = srgan_generator((16, 16, 3), input_=generator_in)

discriminator_in_predicted = Input(shape=(32, 32, 3))
discriminator_in_original = Input(shape=(32, 32, 3))
model_discriminator = srgan_discriminator(discriminator_in_predicted, discriminator_in_original)

generator_out = model_generator(generator_in)
discriminator_out = model_discriminator([generator_out, discriminator_in_original])
model_srgan = Model([generator_in, discriminator_in_original], outputs=discriminator_out)

In [ ]:
model_generator.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model_discriminator.compile(optimizer=adam(lr=0.00002), loss='binary_crossentropy', metrics=['accuracy'])
model_srgan.compile(optimizer=adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_srgan.summary()

In [ ]:
epochs = 20
batch_size = 256
train_version = 5

In [ ]:
model_generator.load_weights(path_data + ('srgan_generator_%s.model' % train_version))
model_discriminator.load_weights(path_data + ('srgan_discriminator_%s.model' % train_version))

In [ ]:
checkpointer = ModelCheckpoint(filepath='data/srgan_weights_%s.{epoch:03d}_{val_acc:.4f}.hdf5' % 
                               train_version, verbose=1, save_best_only=False)
tb_callback = TensorBoard(log_dir='data/tensorboard/', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
# model_generator.fit(x_train_lr, x_train, epochs=100, validation_data=(x_test_lr, x_test), callbacks=[tb_callback])

# discriminative network should be online trained while training whole GAN as well, so that 
# it improves on differating the original vs superscaled

def set_trainable(model, key_word, value=True):
    layers_list = [layer for layer in model.layers if key_word in layer.name]
    for layer in layers_list:
        layer.trainable = value

metrics_names = model_discriminator.metrics_names
with tf.device('/gpu:0'):
    for epoch in range(epochs):
        print()
        print('Epoch %s / %s' % (epoch, epochs))
        
        with tqdm(total=x_train_lr.shape[0] // batch_size) as progress:
            for batch_i in np.arange(0, x_train_lr.shape[0], batch_size):
                if batch_size + batch_i > x_train_lr.shape[0]:
                    continue

                x_train_batch = x_train_norm[batch_i:batch_i+batch_size]
                x_train_lr_batch = x_train_lr_norm[batch_i:batch_i+batch_size]
                x_train_predictions = model_generator.predict(x_train_lr_batch)

                set_trainable(model_discriminator, 'discriminator', False)
                metrics_srgan = model_srgan.train_on_batch([x_train_lr_batch, x_train_batch], [1] * batch_size)
            
                set_trainable(model_discriminator, 'discriminator', True)
                metrics = model_discriminator.train_on_batch([np.concatenate([x_train_predictions, x_train_batch]),
                                                              np.concatenate([x_train_batch, x_train_batch])], 
                                                             np.concatenate([[0] * batch_size, [1] * batch_size]))
                progress.set_description('Srgan: %s: %s; %s: %s; Discriminator: %s: %s; %s: %s' % 
                    (metrics_names[0], metrics_srgan[0], metrics_names[1], metrics_srgan[1], metrics_names[0], 
                     metrics[0], metrics_names[1], metrics[1]))
                progress.update()

            eval_srgan = model_srgan.evaluate([x_test_lr_norm, x_test_norm], [1] * x_test_lr_norm.shape[0], verbose=0)
            print('Model SRGAN evaluation: %s: %s; %s: %s;' % (metrics_names[0], eval_srgan[0], metrics_names[1], 
                                                               eval_srgan[1]))
            
            eval_g = model_generator.evaluate(x_test_lr_norm, x_test_norm, verbose=0)
            print('Model generator evaluation: %s: %s; %s: %s' % (metrics_names[0], eval_g[0], metrics_names[1], 
                                                                  eval_g[1]))

In [ ]:
predictions = model_generator.predict(x_train_lr_norm[:1])

display(Image.fromarray(x_train[0]).resize((128, 128)))
display(Image.fromarray((x_train_lr[0] * 255).astype(np.uint8)).resize((128, 128)))
display(Image.fromarray((predictions[0] * x_train_std + x_train_mean).astype(np.uint8)).resize((128, 128)))

In [ ]:
print(model_srgan.evaluate(x_test_lr, [1] * x_test_lr.shape[0]))

In [ ]:
model_srgan.save(path_data + ('srgan_%s.model' % train_version))
model_generator.save(path_data + ('srgan_final_generator_%s.model' % train_version))
model_discriminator.save(path_data + ('srgan_final_discriminator_%s.model' % train_version))

In [ ]:
for x, x_lr, type_ in [(x_train, x_train_lr_norm, 'train'), (x_test, x_test_lr_norm, 'test')]:
    plt.figure(figsize=(15, 30))
    for i in range(10):
        plt.subplot(10, 3, (i * 3) + 1)
        plt.imshow(((x_lr[i] * x_train_lr_std + x_train_lr_mean) * 255).astype(np.uint8))

        plt.subplot(10, 3, (i * 3) + 2)
        plt.imshow(x[i])

        plt.subplot(10, 3, (i * 3) + 3)
        h, w = x_lr.shape[1], x_lr.shape[2]
        prediction = model_generator.predict(x_lr[i].reshape(1, h, w, 3))[0]
        plt.imshow((prediction * x_train_std + x_train_mean).astype(np.uint8))

    plt.savefig('data/srgan_final_%s_%s.png' % (train_version, type_))
    plt.show()

In [ ]:
display(Image.fromarray(np.uint8((prediction - np.min(prediction))*255/(np.max(prediction) - np.min(prediction))))\
        .resize((128, 128)))
display(Image.fromarray((prediction * x_train_std + x_train_mean).astype(np.uint8)).resize((128, 128)))

In [ ]:
# model_generator.fit(x_train_lr, x_train, epochs=100, validation_data=(x_test_lr, x_test), callbacks=[tb_callback])

# discriminative network should be online trained while training whole GAN as well, so that 
# it improves on differating the original vs superscaled

def set_trainable(model, key_word, value=True):
    layers_list = [layer for layer in model.layers if key_word in layer.name]
    for layer in layers_list:
        layer.trainable = value

metrics_names = model_discriminator.metrics_names
with tf.device('/gpu:0'):
    for _ in range(30):
        with tqdm(total=x_train_lr.shape[0] // batch_size) as progress:
            for batch_i in np.arange(0, x_train_lr.shape[0], batch_size):
                if batch_size + batch_i > x_train_lr.shape[0]:
                    continue

                x_train_batch = x_train_norm[batch_i:batch_i+batch_size]
                x_train_lr_batch = x_train_lr_norm[batch_i:batch_i+batch_size]

    #             set_trainable(model_generator, 'generator', True)
                set_trainable(model_discriminator, 'discriminator', False)

                metrics_srgan = model_srgan.train_on_batch(x_train_lr_batch, [1] * batch_size)
            
    #             set_trainable(model_generator, 'generator', False)
                set_trainable(model_discriminator, 'discriminator', True)

                x_train_predictions = model_generator.predict(x_train_lr_batch)
                metrics = model_discriminator.train_on_batch(np.concatenate([x_train_predictions, x_train_batch]), 
                                                             np.concatenate([[0] * batch_size, [1] * batch_size]))
                progress.set_description('Srgan: %s: %s; %s: %s; Discriminator: %s: %s; %s: %s' % 
                    (metrics_names[0], metrics_srgan[0], metrics_names[1], metrics_srgan[1], metrics_names[0], 
                     metrics[0], metrics_names[1], metrics[1]))
                progress.update()

#             print(model_srgan.evaluate(x_test_lr_norm, [1] * x_test_lr_norm.shape[0]))
#             print(model_generator.evaluate(x_test_lr_norm, x_test_norm))

In [ ]:
np.min(model_generator.predict(x_lr[i].reshape(1, h, w, 3)))

In [ ]:
model_srgan.predict(np.zeros((1, 16, 16, 3)))

In [ ]:
Image.fromarray(model_generator.predict(x_lr[i].reshape(1, h, w, 3)).astype(np.uint8)[0])

In [ ]:
x_lr[0]

In [ ]:
x_train[0].reshape(1, 32, 32, 3).shape

In [ ]:
model_discriminator.predict(model_generator.predict(x_train_lr[0].reshape(1, 16, 16, 3)))

In [ ]:
x_lr[i].reshape(1, h, w, 3)

In [ ]:
model_generator.predict(x_lr[i].reshape(1, h, w, 3).astype(np.uint8)).astype(np.uint8)[0]

In [ ]:
model_generator.predict(x_lr[0].reshape(1, h, w, 3))

In [ ]:
model_generator.predict(x_lr[0].reshape(1, h, w, 3)).astype(np.uint8)